In [ ]:
# 概率人体健康风险评估 (Probabilistic Human Health Risk Assessment)
import os
import pandas as pd
import numpy as np

# 计算 口服摄入 暴露剂量 (Calculate oral ingestion exposure dose)
def cal_add_oral_ingestion(C_i, IR, EF, ED, BW, AT):
    return (C_i/1000 * IR * EF * ED) / (BW * AT)

# 计算 吸入 暴露剂量 (Calculate inhalation exposure dose)
def cal_add_inhalation(C_i, DAIR, EF, ED, VF, BW, AT):
    return (C_i/1000 * DAIR * EF * ED * VF) / (BW * AT)

# 计算 皮肤接触 暴露剂量 (Calculate dermal contact exposure dose)
def cal_add_dermal(C_i, SA, F, PC, ET, EF, ED, BW, AT):
    return (C_i/1000 * SA * F * PC * ET * EF * ED) / (BW * AT) * 1e-3

# 计算 致癌风险 (Calculate cancer risk)
def cal_CR(add, SF):
    return add * SF

# 计算 非致癌风险 (Calculate non-cancer risk)
def cal_hq(add, RfD):
    RfD[RfD == 0] = np.nan  # RfD里有0 (RfD contains 0)
    return add / RfD

In [ ]:

# 读取污染物参数 (Read pollutant parameters)
file_root_dir = os.path.dirname(os.path.realpath('__file__'))
file_path = os.path.join(file_root_dir, 'data', 'Pollutants toxicity.xlsx')
param_data = pd.read_excel(file_path, index_col=0)
param_data = param_data.drop(param_data.index[0])
print(param_data.tail())

labels = ['(0-5) years', '(6-10) years', '(11-18) years', '(19-70) years']
iloc_col = {  # (0-5)
    'IR': 6, 
    'EF': 10, 
    'ED': 14, 
    'BW': 18, 
    'AT': 22, 
    'DAIR': 35, 
    'VF': 47, 
    'SA': 68, 
    'F': 72, 
    'PC': 76, 
    'ET': 80, 
    
    'SF_o': 26, 
    'RfD_o': 30,
    'SF_i': 59, 
    'RfD_i': 63, 
    'SF_d': 100, 
    'RfD_d': 104, 
}

# 蒙特卡洛随机区间 (Monte Carlo random intervals)
exposure_data = pd.read_excel(os.path.join(file_root_dir, 'data', 'pollutant exposure parameters.xlsx'), index_col=0)
print(exposure_data.tail())

# 读取污染物浓度 (Read pollutant concentrations)
conc_name = 'pollutant_value-combine_reorganizes_Mean'
conc_data = pd.read_excel(os.path.join(file_root_dir, 'data', f'{conc_name}.xlsx'), index_col=0)
print(conc_data.tail())

In [ ]:
# 蒙特卡洛 样本数 (Monte Carlo sample size)
num_simulations = 10000

poll_len = int(len(param_data.index))
total_len = int(len(conc_data.index))
point_len = int(total_len / len(param_data.index))

for idx in range(len(labels)):
    label = labels[idx]
    # C_i = conc_data.loc[:, 'Concentration(ug/L)'].values
    C_i = conc_data.loc[:, 'Mean_Concentration(ug/L)'].values.astype(np.float32)  # 改为 float32 (Change to float32)

    # # 标准 (Standards)
    IR = param_data.iloc[:, iloc_col['IR'] + idx].values
    EF = param_data.iloc[:, iloc_col['EF'] + idx].values
    ED = param_data.iloc[:, iloc_col['ED'] + idx].values
    BW = param_data.iloc[:, iloc_col['BW'] + idx].values
    AT = param_data.iloc[:, iloc_col['AT'] + idx].values
    DAIR = param_data.iloc[:, iloc_col['DAIR'] + idx].values
    VF = param_data.iloc[:, iloc_col['VF'] + idx].values
    SA = param_data.iloc[:, iloc_col['SA'] + idx].values
    F = param_data.iloc[:, iloc_col['F'] + idx].values
    PC = param_data.iloc[:, iloc_col['PC'] + idx].values
    ET = param_data.iloc[:, iloc_col['ET'] + idx].values

    SF_o = param_data.iloc[:, iloc_col['SF_o'] + idx].values
    RfD_o = param_data.iloc[:, iloc_col['RfD_o'] + idx].values
    SF_i = param_data.iloc[:, iloc_col['SF_i'] + idx].values
    RfD_i = param_data.iloc[:, iloc_col['RfD_i'] + idx].values
    SF_d = param_data.iloc[:, iloc_col['SF_d'] + idx].values
    RfD_d = param_data.iloc[:, iloc_col['RfD_d'] + idx].values
    
    # 剂量计算 (Dose calculation)
    add_oral_ingestion = cal_add_oral_ingestion(
        C_i=C_i, 
        IR=np.tile(IR, point_len), 
        EF=np.tile(EF, point_len), 
        ED=np.tile(ED, point_len), 
        BW=np.tile(BW, point_len), 
        AT=np.tile(AT, point_len), 
    )
    
    add_dermal = cal_add_dermal(
        C_i=C_i, 
        SA=np.tile(SA, point_len), 
        F=np.tile(F, point_len), 
        PC=np.tile(PC, point_len), 
        ET=np.tile(ET, point_len), 
        EF=np.tile(EF, point_len), 
        ED=np.tile(ED, point_len), 
        BW=np.tile(BW, point_len), 
        AT=np.tile(AT, point_len), 
    )
    
    add_inhalation = cal_add_inhalation(
        C_i=C_i, 
        DAIR=np.tile(DAIR, point_len), 
        EF=np.tile(EF, point_len), 
        ED=np.tile(ED, point_len), 
        VF=np.tile(VF, point_len), 
        BW=np.tile(BW, point_len), 
        AT=np.tile(AT, point_len), 
    )
    
    # 致癌风险 (Cancer risk)
    CR_oral_ingestion = cal_CR(add=add_oral_ingestion, SF=np.tile(SF_o, point_len))
    CR_dermal = cal_CR(add=add_dermal, SF=np.tile(SF_d, point_len))
    CR_inhalation = cal_CR(add=add_inhalation, SF=np.tile(SF_i, point_len))
    # 非致癌风险 (Non-cancer risk)
    HQ_oral_ingestion = cal_hq(add=add_oral_ingestion, RfD=np.tile(RfD_o, point_len))
    HQ_dermal = cal_hq(add=add_dermal, RfD=np.tile(RfD_d, point_len))
    HQ_inhalation = cal_hq(add=add_inhalation, RfD=np.tile(RfD_i, point_len))
        
    # # 蒙特卡洛 (Monte Carlo)
    # 固定参数的概率分布 - 均匀分布 (Probability distribution of fixed parameters - uniform distribution)
    IR_std, IR_mean = exposure_data.loc['GW ingestion rate (IR)', f'{label}_2'], exposure_data.loc['GW ingestion rate (IR)', f'{label}_1']
    IR = np.random.lognormal(IR_mean, IR_std, num_simulations)  # 水摄入率的对数正态分布（L/d） IR (Normal distribution of water intake rate (L/d) IR)
    EF = exposure_data.loc['Exposure frequency (EF)', f'{label}_2']   # 暴露频率（天/年）     EF (Exposure frequency (days/year) EF)
    ED_min, ED_max = exposure_data.loc['Exposure duration (ED)', f'{label}_2'], exposure_data.loc['Exposure duration (ED)', f'{label}_1'] 
    ED = np.random.uniform(ED_min, ED_max, num_simulations)  # 暴露持续时间（年）    ED (Exposure duration (years) ED)
    BW_std, BW_mean = exposure_data.loc['Body weight (BW)', f'{label}_2'], exposure_data.loc['Body weight (BW)', f'{label}_1']                 
    BW = np.random.normal(BW_mean, BW_std, num_simulations)  # 体重（kg） BW (Body weight (kg) BW)
    AT_C = exposure_data.loc['Averaging Time –Cancer risk', f'{label}_2']   # 致癌平均暴露时间     AT (Cancer risk averaging time AT)
    AT_NC = exposure_data.loc['Averaging Time - non cancer risk', f'{label}_2']   # 非致癌平均暴露时间     AT (Non-cancer risk averaging time AT)
    DAIR = exposure_data.loc['Daily air pulmonary ventilation rate (DAIR)', f'{label}_2']   # 每日空气肺通风率     DAIR (Daily air pulmonary ventilation rate DAIR)
    VF = exposure_data.loc['Volatile factor (VF)', f'{label}_2']   # 挥发性因子     VF (Volatile factor VF)
    SA_std, SA_mean = exposure_data.loc['Skin area (SA)', f'{label}_2'], exposure_data.loc['Skin area (SA)', f'{label}_1']
    SA = np.random.lognormal(SA_mean, SA_std, num_simulations)  # 皮肤暴露面积的对数正态分布（cm2）SA (Normal distribution of skin exposure area (cm2) SA)
    F_min, F_max = exposure_data.loc['Fraction of surface skin in contact with water (F)', f'{label}_2'], exposure_data.loc['Fraction of surface skin in contact with water (F)', f'{label}_1']
    F = np.random.uniform(F_min, F_max, num_simulations)  # 与水接触的表面皮肤的比例  F (Proportion of surface skin in contact with water F)
    PC = exposure_data.loc['Permeability constant (PC)', f'{label}_2']   # 皮肤渗透系数（cm/h） PC (Skin permeability constant (cm/h) PC)
    ET_std, ET_mean = exposure_data.loc['Exposure time (ET)', f'{label}_2'], exposure_data.loc['Exposure time (ET)', f'{label}_1']
    ET = np.random.lognormal(ET_mean, ET_std, num_simulations)  # 皮肤暴露时间（h/day） ET (Skin exposure time (h/day) ET)
    
    # 剂量计算 (Dose calculation)
    C_i_simulation = np.tile(C_i[:, np.newaxis], (1, num_simulations))
    # # 浓度随机50% (50% random concentration)
    # C_i_simulation = C_i_simulation - np.random.random(C_i_simulation.shape) * 0.9 * C_i_simulation
    
    add_oral_ingestion_C = cal_add_oral_ingestion(
        C_i=C_i_simulation, 
        IR=IR, 
        EF=EF, 
        ED=EF, 
        BW=BW, 
        AT=AT_C, 
    )
    
    add_oral_ingestion_NC = cal_add_oral_ingestion(
        C_i=C_i_simulation, 
        IR=IR, 
        EF=EF, 
        ED=EF, 
        BW=BW, 
        AT=AT_NC, 
    )
    
    add_dermal_C = cal_add_dermal(
        C_i=C_i_simulation, 
        SA=SA, 
        F=F, 
        PC=PC, 
        ET=ET, 
        EF=EF, 
        ED=ED, 
        BW=BW, 
        AT=AT_C, 
    )
    
    add_dermal_NC = cal_add_dermal(
        C_i=C_i_simulation, 
        SA=SA, 
        F=F, 
        PC=PC, 
        ET=ET, 
        EF=EF, 
        ED=ED, 
        BW=BW, 
        AT=AT_NC, 
    )
    
    add_inhalation_C = cal_add_inhalation(
        C_i=C_i_simulation, 
        DAIR=DAIR, 
        EF=EF, 
        ED=ED, 
        VF=VF, 
        BW=BW, 
        AT=AT_C, 
    )
    
    add_inhalation_NC = cal_add_inhalation(
        C_i=C_i_simulation, 
        DAIR=DAIR, 
        EF=EF, 
        ED=ED, 
        VF=VF, 
        BW=BW, 
        AT=AT_C, 
    )
    
    # 致癌 风险 (Cancer risk)
    CR_oral_ingestion_C = cal_CR(add=add_oral_ingestion_C, SF=np.array([1.0]))
    CR_dermal_C = cal_CR(add=add_dermal_C, SF=np.array([1.0]))
    CR_inhalation_C = cal_CR(add=add_inhalation_C, SF=np.array([1.0]))
    # 非致癌 风险 (Non-cancer risk)
    HQ_oral_ingestion_NC = cal_hq(add=add_oral_ingestion_NC, RfD=np.array([1.0]))
    HQ_dermal_C_NC = cal_hq(add=add_dermal_NC, RfD=np.array([1.0]))
    HQ_inhalation_NC = cal_hq(add=add_inhalation_NC, RfD=np.array([1.0]))
    # 总健康风险叠加 (Total health risk superposition)
    HQ_total = HQ_oral_ingestion_NC + HQ_dermal_C_NC + HQ_inhalation_NC  # shape: (污染物数, 样本数) (shape: (pollutant count, sample count))
    CR_total = CR_oral_ingestion_C + CR_dermal_C + CR_inhalation_C       # shape: (污染物数, 样本数) (shape: (pollutant count, sample count))

    # 计算超标概率 (Calculate exceedance probability)
    HQ_prob_exceed_1 = np.mean(HQ_total > 1, axis=1)  # 每个污染物 HQ 超过 1 的概率 (Probability of HQ exceeding 1 for each pollutant)
    CR_prob_exceed_1e4 = np.mean(CR_total > 1e-4, axis=1)  # 每个污染物 CR 超过 1E-4 的概率 (Probability of CR exceeding 1E-4 for each pollutant)

    # 风险数据整理 (Risk data organization)
    risk = pd.DataFrame(
        np.column_stack([
            conc_data.index, conc_data.loc[:, 'CAS'], conc_data.loc[:, 'Pollutants_name'], conc_data.loc[:, 'Abbreviation'], conc_data.loc[:, '污染物名称'], conc_data.loc[:, 'Class'], 
            CR_oral_ingestion, CR_dermal, CR_inhalation, HQ_oral_ingestion, HQ_dermal, HQ_inhalation, 
            np.mean(CR_oral_ingestion_C, axis=1), np.percentile(a=CR_oral_ingestion_C, q=95, axis=1), 
            np.mean(CR_dermal_C, axis=1), np.percentile(a=CR_dermal_C, q=95, axis=1), 
            np.mean(CR_inhalation_C, axis=1), np.percentile(a=CR_inhalation_C, q=95, axis=1), 
            np.mean(HQ_oral_ingestion_NC, axis=1), np.percentile(a=HQ_oral_ingestion_NC, q=95, axis=1), 
            np.mean(HQ_dermal_C_NC, axis=1), np.percentile(a=HQ_dermal_C_NC, q=95, axis=1), 
            np.mean(HQ_inhalation_NC, axis=1), np.percentile(a=HQ_inhalation_NC, q=95, axis=1), 
            CR_prob_exceed_1e4, HQ_prob_exceed_1
        ]), 
        columns=[
            'Point', 'CAS', 'Pollutants_name', 'Abbreviation', '污染物名称', 'Class',
            'CR_oral_ingestion', 'CR_dermal', 'CR_inhalation', 'HQ_oral_ingestion', 'HQ_dermal', 'HQ_inhalation', 
            'CR_oral_ingestion_mean', 'CR_oral_ingestion_95th', 
            'CR_dermal_mean', 'CR_dermal_95th', 
            'CR_inhalation_mean', 'CR_inhalation_95th', 
            'HQ_oral_ingestion_mean', 'HQ_oral_ingestion_95th', 
            'HQ_dermal_mean', 'HQ_dermal_95th', 
            'HQ_inhalation_mean', 'HQ_inhalation_95th', 
            'CR_prob_exceed_1e4', 'HQ_prob_exceed_1'
        ]
    )
    risk.to_excel(f'{conc_name}_{label}-输出超标概率.xlsx') # 保存结果到Excel文件 (Save results to Excel file)